In [1]:
import numpy as np
import tensorflow as tf
tf.random.set_seed(42)

In [2]:
scalar = tf.constant(1, dtype=tf.int8)
vector = tf.constant([1, 2, 3], dtype=tf.float32)
matrix = tf.constant(np.array([[1, 2], [3, 4]]), dtype=tf.float32)
print([scalar, vector, matrix])

[<tf.Tensor: shape=(), dtype=int8, numpy=1>, <tf.Tensor: shape=(3,), dtype=float32, numpy=array([1., 2., 3.], dtype=float32)>, <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1., 2.],
       [3., 4.]], dtype=float32)>]


In [3]:
# Get shape and object shape
print(scalar.shape)
print(tf.shape(scalar))
print(id(scalar))

print(vector.shape)
print(tf.shape(vector))
print(id(vector))

print(matrix.shape)
print(tf.shape(matrix))
print(id(matrix))

()
tf.Tensor([], shape=(0,), dtype=int32)
2060390467712
(3,)
tf.Tensor([3], shape=(1,), dtype=int32)
2060390467904
(2, 2)
tf.Tensor([2 2], shape=(2,), dtype=int32)
2060390468096


In [4]:
# Convert data type
matrix = tf.cast(matrix, dtype=tf.int8)
print(matrix)
print(id(matrix))
# Notice the new tensor has the same values as the original tensor, but with int8 data type
# and it also has a different id number 4, indicating that it is a new tensor object.

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int8)
2060390468288


In [5]:
# Ways to create special tensor
o = tf.zeros((2, 2))
x = tf.random.uniform((3, 2))
print(o)
print(x)
print(tf.ones_like(o))

tf.Tensor(
[[0. 0.]
 [0. 0.]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[0.6645621  0.44100678]
 [0.3528825  0.46448255]
 [0.03366041 0.68467236]], shape=(3, 2), dtype=float32)
tf.Tensor(
[[1. 1.]
 [1. 1.]], shape=(2, 2), dtype=float32)


In [6]:
# We saw earlier that we can convert numpy arrays to Tensors with tf.constant , we can
# do the reverse with .numpy() method
numpy_matrix = matrix.numpy()
type(numpy_matrix)
numpy_matrix

array([[1, 2],
       [3, 4]], dtype=int8)

## Operations

In [7]:
print(matrix[0, 1])
print(matrix[1, :2])
print(matrix[tf.newaxis, 1, :2])

tf.Tensor(2, shape=(), dtype=int8)
tf.Tensor([3 4], shape=(2,), dtype=int8)
tf.Tensor([[3 4]], shape=(1, 2), dtype=int8)


In [8]:
# Addition
a = tf.constant([1, 2, 3], dtype=tf.float32)
b = tf.constant([4, 5, 6], dtype=tf.float32)
print(tf.add(a, b))

# Multiplication
print(tf.multiply(a, b))

# Matrix - Multiplication
print(tf.multiply(matrix, matrix))

tf.Tensor([5. 7. 9.], shape=(3,), dtype=float32)
tf.Tensor([ 4. 10. 18.], shape=(3,), dtype=float32)
tf.Tensor(
[[ 1  4]
 [ 9 16]], shape=(2, 2), dtype=int8)


## Variables

In [9]:
v = tf.Variable(x)
print(v)

<tf.Variable 'Variable:0' shape=(3, 2) dtype=float32, numpy=
array([[0.6645621 , 0.44100678],
       [0.3528825 , 0.46448255],
       [0.03366041, 0.68467236]], dtype=float32)>


In [10]:
# Variables can be inputs to operations just as Tensors, note the output is a Tensor not Variable.
print(tf.square(v))

tf.Tensor(
[[0.4416428  0.19448698]
 [0.12452606 0.21574403]
 [0.00113302 0.46877623]], shape=(3, 2), dtype=float32)


In [11]:
# Variables can be updated with .assign , .assign_add or .assign_sub methods.
v.assign(tf.square(v))
print(v)

<tf.Variable 'Variable:0' shape=(3, 2) dtype=float32, numpy=
array([[0.4416428 , 0.19448698],
       [0.12452606, 0.21574403],
       [0.00113302, 0.46877623]], dtype=float32)>


## Automatic Differentiation

In [16]:
a = tf.Variable([4], dtype=tf.float32)
b = tf.Variable([5], dtype=tf.float32)
def f(a, b, power=2, d=3):
    return tf.pow(a, power) + d * b

with tf.GradientTape(watch_accessed_variables=True) as tape:
    c = f(a, b)
print(tape.gradient(target=c, sources=[a, b]))
print(f(a, b))

[<tf.Tensor: shape=(1,), dtype=float32, numpy=array([8.], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([3.], dtype=float32)>]
tf.Tensor([31.], shape=(1,), dtype=float32)


## Linear Regression

In [27]:
# ground truth
true_weights = tf.constant(list(range(5)), dtype=tf.float32)[:, tf.newaxis]

# some random training data
x = tf.constant(tf.random.uniform((32, 5)), dtype=tf.float32)
y = tf.constant(x @ true_weights, dtype=tf.float32)

# model parameters
weights = tf.Variable(tf.random.uniform((5, 1)), dtype=tf.float32)
for iteration in range(1001):
    with tf.GradientTape() as tape:
        y_hat = tf.linalg.matmul(x, weights)
        loss = tf.reduce_mean(tf.square(y - y_hat))
    if not (iteration % 100):
        print('mean squared loss at iteration {:4d} is {:5.4f}'.format(iteration, loss))
    gradients = tape.gradient(loss, weights)
    weights.assign_add(-0.05 * gradients)
print(weights)

mean squared loss at iteration    0 is 18.2589
mean squared loss at iteration  100 is 0.1475
mean squared loss at iteration  200 is 0.0290
mean squared loss at iteration  300 is 0.0062
mean squared loss at iteration  400 is 0.0014
mean squared loss at iteration  500 is 0.0003
mean squared loss at iteration  600 is 0.0001
mean squared loss at iteration  700 is 0.0000
mean squared loss at iteration  800 is 0.0000
mean squared loss at iteration  900 is 0.0000
mean squared loss at iteration 1000 is 0.0000
<tf.Variable 'Variable:0' shape=(5, 1) dtype=float32, numpy=
array([[4.5362278e-04],
       [1.0005250e+00],
       [2.0009515e+00],
       [2.9989629e+00],
       [3.9992294e+00]], dtype=float32)>
